<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_2C_DataAggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

References


*   https://www.enterprisedb.com/postgres-tutorials/how-use-grouping-sets-cube-and-rollup-postgresql

Free Data Sets


1.   https://www.iguazio.com/blog/13-best-free-retail-datasets-for-machine-learning/






In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-03-11 06:51:09.476437+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Setup

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#
import psycopg2
import pandas as pd
#
!psql -V
!service postgresql start





psql (PostgreSQL) 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


## Prepare FACT table

In [4]:
# Create Fact Table using command line interface
#
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c \
"drop table if exists ss_order;\
CREATE TABLE IF NOT EXISTS ss_order (\
    RowID smallint,\
    OrderID char(14),OrderDate date,\
    ShipDate date,ShipMode varchar(16),\
    CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
    Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
    ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
    Sales decimal(8,2), Quantity smallint, Discount decimal(4,2), Profit decimal(8,2) \
); \
"

NOTICE:  table "ss_order" does not exist, skipping
CREATE TABLE


In [5]:
#Describe the table ss_order
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\d+ ss_order'

                                                   Table "public.ss_order"
    Column    |          Type          | Collation | Nullable | Default | Storage  | Compression | Stats target | Description 
--------------+------------------------+-----------+----------+---------+----------+-------------+--------------+-------------
 rowid        | smallint               |           |          |         | plain    |             |              | 
 orderid      | character(14)          |           |          |         | extended |             |              | 
 orderdate    | date                   |           |          |         | plain    |             |              | 
 shipdate     | date                   |           |          |         | plain    |             |              | 
 shipmode     | character varying(16)  |           |          |         | extended |             |              | 
 customerid   | character(8)           |           |          |         | extended |            

In [6]:
#Get Data File
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [7]:
# Date Format Issues
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "set datestyle to PostgreSQL,European; show datestyle"

   DateStyle   
---------------
 Postgres, DMY
(1 row)



In [8]:
# Load Data
# Need to use U = postgres and provide password = 'pass'
!psql -h localhost -p 5432 -U postgres -d dbpraxis -c \
"set datestyle to PostgreSQL,European;copy ss_order FROM '/content/SS_Orders.csv' DELIMITER ',' CSV HEADER;"

COPY 9994


In [9]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select * from ss_order limit 10'
#the date data should have loaded correctly

 rowid |    orderid     | orderdate  |  shipdate  |    shipmode    | customerid |  customername   |  segment  |    country    |      city       |   state    | postalcode | region |    productid    |    category     | subcategory |                           productname                            | sales  | quantity | discount | profit  
-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------
     1 | CA-2016-152156 | 2016-11-08 | 2016-11-11 | Second Class   | CG-12520   | Claire Gute     | Consumer  | United States | Henderson       | Kentucky   | 42420      | South  | FUR-BO-10001798 | Furniture       | Bookcases   | Bush Somerset Collection Bookcase                                | 261.96 |        2 |     0.

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [10]:
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return


In [11]:
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [12]:
runSelect('select * from ss_order limit 3')

rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,state,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.00,219.58
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.00,6.87


#Group By - Basic

In [13]:
sql = "\
SELECT segment, region, category, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region, category; \
"
runSelect(sql)

segment,region,category,sales
Corporate,South,Office Supplies,45930.19
Consumer,South,Technology,65276.15
Corporate,East,Technology,69725.58
Corporate,East,Office Supplies,66474.79
Consumer,East,Furniture,114211.77
Consumer,West,Furniture,119808.16
Home Office,West,Furniture,49724.57
Corporate,South,Technology,46310.75
Home Office,East,Furniture,29870.32
Consumer,Central,Office Supplies,93111.40


In [14]:
sql = "\
SELECT segment, region, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region; \
"
runSelect(sql)

segment,region,sales
Home Office,Central,91212.66
Corporate,West,225855.25
Home Office,South,74255.03
Consumer,Central,252031.43
Home Office,West,136721.86
Corporate,South,121885.95
Consumer,South,195580.92
Corporate,Central,157995.79
Consumer,West,362880.82
Consumer,East,350908.17


In [15]:
sql = "\
SELECT segment, SUM (sales) sales \
FROM ss_order \
GROUP BY segment; \
"
runSelect(sql)

segment,sales
Consumer,1161401.34
Corporate,706146.44
Home Office,429653.29


In [16]:
sql = "\
SELECT  SUM (sales) sales \
FROM ss_order  \
"
runSelect(sql)

sales
2297201.07


#Roll Up

## Basic Roll up

In [17]:
sql = "\
SELECT category, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
runSelect(sql)

category,sum
None,37873
Furniture,8028
Office Supplies,22906
Technology,6939


In [18]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
runSelect(sql)

category,sum
All category,37873
Furniture,8028
Office Supplies,22906
Technology,6939


## MultiColumn Roll Up

In [19]:
 sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (category , segment); \
"
runSelect(sql)

category,segment,quantity
None,None,37873
Office Supplies,Corporate,7018
Furniture,Corporate,2495
Office Supplies,Home Office,4130
Technology,Consumer,3597
Technology,Corporate,2095
Office Supplies,Consumer,11758
Technology,Home Office,1247
Furniture,Home Office,1367
Furniture,Consumer,4166


In [20]:
# note change in order of ROLLUP
sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
runSelect(sql)

category,segment,quantity
None,None,37873
Office Supplies,Home Office,4130
Office Supplies,Consumer,11758
Technology,Corporate,2095
Office Supplies,Corporate,7018
Furniture,Home Office,1367
Technology,Home Office,1247
Furniture,Corporate,2495
Furniture,Consumer,4166
Technology,Consumer,3597


In [21]:
sql ="\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
runSelect(sql)

category,segment,quantity
All category,All segment,37873
Office Supplies,Home Office,4130
Office Supplies,Consumer,11758
Technology,Corporate,2095
Office Supplies,Corporate,7018
Furniture,Home Office,1367
Technology,Home Office,1247
Furniture,Corporate,2495
Furniture,Consumer,4166
Technology,Consumer,3597


## Partial Roll Up

In [22]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY segment, ROLLUP (category); \
"
runSelect(sql)

category,segment,quantity
Office Supplies,Home Office,4130
Office Supplies,Consumer,11758
Technology,Corporate,2095
Office Supplies,Corporate,7018
Furniture,Home Office,1367
Technology,Home Office,1247
Furniture,Corporate,2495
Furniture,Consumer,4166
Technology,Consumer,3597
All category,Consumer,19521


In [23]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY category, ROLLUP (segment); \
"
runSelect(sql)

category,segment,quantity
Office Supplies,Corporate,7018
Furniture,Corporate,2495
Office Supplies,Home Office,4130
Technology,Consumer,3597
Technology,Corporate,2095
Office Supplies,Consumer,11758
Technology,Home Office,1247
Furniture,Home Office,1367
Furniture,Consumer,4166
Furniture,All segment,8028


#Cube

In [24]:
#One Dimensional Cube
#
sql = "\
SELECT COALESCE(category, 'All category') category, SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category) \
ORDER BY category;  \
"
runSelect(sql)

category,quantity
All category,37873
Furniture,8028
Office Supplies,22906
Technology,6939


In [25]:
#Two Dimensional Cube
#
sql = "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment) \
ORDER BY category,segment;  \
"
runSelect(sql)

category,segment,quantity
All category,All segment,37873
All category,Consumer,19521
All category,Corporate,11608
All category,Home Office,6744
Furniture,All segment,8028
Furniture,Consumer,4166
Furniture,Corporate,2495
Furniture,Home Office,1367
Office Supplies,All segment,22906
Office Supplies,Consumer,11758


In [26]:
# Three Dimensional Cube
#
sql = "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
COALESCE(region, 'All region') region, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment, region) \
ORDER BY category,segment, region;  \
"
runSelect(sql)

category,segment,region,quantity
All category,All segment,All region,37873
All category,All segment,Central,8780
All category,All segment,East,10618
All category,All segment,South,6209
All category,All segment,West,12266
All category,Consumer,All region,19521
All category,Consumer,Central,4519
All category,Consumer,East,5347
All category,Consumer,South,3178
All category,Consumer,West,6477


# Grouping Sets

In [27]:
#Rather confusing
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c  "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment) \
    ); \
"

    category     | region  |   segment   | qty  
-----------------+---------+-------------+------
 Furniture       | East    | Corporate   |  723
 Furniture       | West    | Home Office |  476
 Office Supplies | South   | Corporate   | 1281
 Technology      | South   | Consumer    |  584
 Furniture       | South   | Corporate   |  415
 Technology      | South   | Corporate   |  320
 Furniture       | South   | Home Office |  181
 Furniture       | Central | Consumer    |  952
 Office Supplies | Central | Consumer    | 2777
 Office Supplies | West    | Home Office | 1301
 Office Supplies | Central | Corporate   | 1605
 Office Supplies | East    | Home Office | 1182
 Office Supplies | East    | Consumer    | 3245
 Technology      | East    | Corporate   |  600
 Technology      | West    | Consumer    | 1235
 Furniture       | East    | Consumer    | 1114
 Office Supplies | West    | Corporate   | 2097
 Furniture       | West    | Consumer    | 1405
 Furniture       | South   | Consumer 

In [28]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c  "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region) \
    ); \
"

    category     | region  | qty  
-----------------+---------+------
 Furniture       | West    | 2696
 Office Supplies | South   | 3800
 Furniture       | East    | 2214
 Office Supplies | Central | 5409
 Technology      | East    | 1942
 Technology      | Central | 1544
 Office Supplies | East    | 6462
 Technology      | West    | 2335
 Technology      | South   | 1118
 Office Supplies | West    | 7235
 Furniture       | Central | 1827
 Furniture       | South   | 1291
(12 rows)



In [29]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c  "\
SELECT category,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category) \
    ); \
"

    category     |  qty  
-----------------+-------
 Furniture       |  8028
 Office Supplies | 22906
 Technology      |  6939
(3 rows)



In [30]:
#Rather confusing
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment), \
        (category,region), \
        (category, segment), \
        (segment, region), \
        (category), \
        (region), \
        (segment), \
        () \
    ); \
"

    category     | region  |   segment   |  qty  
-----------------+---------+-------------+-------
                 |         |             | 37873
 Furniture       | East    | Corporate   |   723
 Furniture       | West    | Home Office |   476
 Office Supplies | South   | Corporate   |  1281
 Technology      | South   | Consumer    |   584
 Furniture       | South   | Corporate   |   415
 Technology      | South   | Corporate   |   320
 Furniture       | South   | Home Office |   181
 Furniture       | Central | Consumer    |   952
 Office Supplies | Central | Consumer    |  2777
 Office Supplies | West    | Home Office |  1301
 Office Supplies | Central | Corporate   |  1605
 Office Supplies | East    | Home Office |  1182
 Office Supplies | East    | Consumer    |  3245
 Technology      | East    | Corporate   |   600
 Technology      | West    | Consumer    |  1235
 Furniture       | East    | Consumer    |  1114
 Office Supplies | West    | Corporate   |  2097
 Furniture       |

In [31]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region), \
        (category), \
        (region), \
        () \
    ); \
"

    category     | region  |  qty  
-----------------+---------+-------
                 |         | 37873
 Furniture       | West    |  2696
 Office Supplies | South   |  3800
 Furniture       | East    |  2214
 Office Supplies | Central |  5409
 Technology      | East    |  1942
 Technology      | Central |  1544
 Office Supplies | East    |  6462
 Technology      | West    |  2335
 Technology      | South   |  1118
 Office Supplies | West    |  7235
 Furniture       | Central |  1827
 Furniture       | South   |  1291
 Furniture       |         |  8028
 Office Supplies |         | 22906
 Technology      |         |  6939
                 | East    | 10618
                 | South   |  6209
                 | West    | 12266
                 | Central |  8780
(20 rows)



#Cross Tab / Pivot Tables

## Level 1

In [32]:
sql = " \
    select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Category \
    "
runSelect(sql)

category,central,east,west,south
Furniture,1827,2214,2696,1291
Office Supplies,5409,6462,7235,3800
Technology,1544,1942,2335,1118


## Level 2

In [33]:
sql = " \
  select \
  COALESCE(Category, 'All Category') AS Category, \
  COALESCE(Region, 'All Region') AS Region, \
  Sum(quantity) Quantity \
  from ss_order \
  group by ROLLUP (Category,Region)  \
"
runSelect(sql)

category,region,quantity
All Category,All Region,37873
Furniture,West,2696
Office Supplies,South,3800
Furniture,East,2214
Office Supplies,Central,5409
Technology,East,1942
Technology,Central,1544
Office Supplies,East,6462
Technology,West,2335
Technology,South,1118


## Level 3 - the FULL CROSSTAB with 3 rotations

In [34]:
sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Central, sums.East, sums.West, sums.South, \
sums.Central + sums.East + sums.West + sums.South  Totals \
FROM ( \
  select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
runSelect(sql)

dim,central,east,west,south,totals
Totals,8780,10618,12266,6209,37873
Furniture,1827,2214,2696,1291,8028
Office Supplies,5409,6462,7235,3800,22906
Technology,1544,1942,2335,1118,6939


In [35]:
 sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Consumer, sums.Corporate, sums.HomeOffice, \
sums.Consumer + sums.Corporate + sums.HomeOffice   Totals \
FROM ( \
  select Category, \
    sum(case when (Segment = 'Consumer') then sales else 0 end) as Consumer, \
    sum(case when (Segment = 'Corporate') then sales else 0 end) as Corporate, \
    sum(case when (Segment = 'Home Office') then sales else 0 end) as HomeOffice \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
runSelect(sql)

dim,consumer,corporate,homeoffice,totals
Totals,1161401.34,706146.44,429653.29,2297201.07
Furniture,391049.43,229019.83,121930.72,741999.98
Office Supplies,363951.96,230676.49,124418.54,719046.99
Technology,406399.95,246450.12,183304.03,836154.10


In [36]:
sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Central, sums.East, sums.West, sums.South, \
sums.Central + sums.East + sums.West + sums.South  Totals \
FROM ( \
  select Category, \
    sum(case when (Region = 'Central') then sales else 0 end) as Central, \
    sum(case when (Region = 'East') then sales else 0 end) as East, \
    sum(case when (Region = 'West') then sales else 0 end) as West, \
    sum(case when (Region = 'South') then sales else 0 end) as South \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
runSelect(sql)

dim,central,east,west,south,totals
Totals,501239.88,678781.36,725457.93,391721.90,2297201.07
Furniture,163797.26,208291.17,252612.87,117298.68,741999.98
Office Supplies,167026.33,205516.15,220853.20,125651.31,719046.99
Technology,170416.29,264974.04,251991.86,148771.91,836154.10


## Verify L3 Pivot with Python Pandas

In [38]:
dfSS = pd.read_csv('SS_Orders.csv', delimiter = ',')
dfSS.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [39]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [42]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [43]:
pd.pivot_table(data=dfSS,index='Category',columns='Segment',values='Sales', aggfunc='sum', margins=True)

Segment,Consumer,Corporate,Home Office,All
Category,,,,
Furniture,391049.31,229019.79,121930.70,741999.80
Office Supplies,363952.14,230676.46,124418.43,719047.03
Technology,406399.90,246450.12,183304.02,836154.03
All,1161401.34,706146.37,429653.15,2297200.86


In [44]:
from datetime import datetime
import pytz
print(datetime.now(pytz.timezone('Asia/Calcutta')))


2024-03-11 07:45:02.479300+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)